In [ ]:
from deva.fileio import load_scenario
import ipywidgets as widgets
from IPython.display import HTML, display
from deva.bounds import tabulate_metrics
import numpy as np


display(HTML('''<style>
    .widget-label { min-width: 20ex !important; }
</style>'''))

In [ ]:
candidates, meta = load_scenario("pareto")
metrics = meta['metrics']
attributes = sorted(list(candidates[0].attributes))

table = tabulate_metrics(candidates, metrics)[attributes]
sign = np.array([-1 if metrics[v]["higherIsBetter"] else 1
                 for v in attributes])
scores = table * sign[None, :]


In [ ]:
def go():
    sliders = []


    manual = list(scores.max())
    priority = list(range(len(attributes)))


    for a in attributes:
        col = np.unique(table[a])
        if metrics[a]["higherIsBetter"]:
            col = col[::-1]
        col = col.astype(int)  # for now
        slider = widgets.SelectionSlider(
            value=col[-1], options=col, description=metrics[a]["name"], readout=True,
            layout=widgets.Layout(width='50%'), continuous_update=True
        )
        sliders.append(slider)



    thresh = manual.copy()
    lock = [False]

    def limit(change):
        
        if not lock[0]:
            # Need to be able to adjust values without triggering
            lock[0] = True
            owner = change['owner']    
            value = change['new']

            ix = sliders.index(owner)  # which slider changed?

            # try to preserve the most-recently-moved
            priority.remove(ix)
            priority.append(ix)

            # remember the value they tried to set
            manual[ix] = sign[ix] * value

            # Now we just go through in order of priority
            active = np.ones(scores.shape[0], dtype=bool)

            # Note - bigger score is worse
            # 
            for p in priority[::-1]:
                col = scores.values[:, p]
                limit = col[active].min()  # best feasible
                # whichever is worse of best feasible and requested
                thresh[p] = max(manual[p], limit)
                active &= (col <= thresh[p])  # filter

            for i in range(len(sliders)):
                if i != ix:
                    sliders[i].value = int(thresh[i]*sign[i])


            lock[0] = False

    for s in sliders:
        s.observe(limit, names='value')


    return widgets.VBox(sliders)

go()

## Get a more intuitive hysteresis...
Perhaps finding the closest point to the starting point?
Don't want the balance to jump around vs number of candidates?

In [ ]:
def go():
    sliders = []
    
    manual = list(scores.max())  # user input

    for a in attributes:
        col = np.unique(table[a])
        if metrics[a]["higherIsBetter"]:
            col = col[::-1]
        col = col.astype(int)  # for now
        slider = widgets.SelectionSlider(
            value=col[-1], options=col, description=metrics[a]["name"], readout=True,
            layout=widgets.Layout(width='50%'), continuous_update=True
        )
        sliders.append(slider)

    lock = [False]

    def limit(change):
        if not lock[0]:
            # Need to be able to adjust values without triggering
            lock[0] = True
            owner = change['owner']    
            value = change['new']
            ix = sliders.index(owner)  # which slider changed?
            # Flip the user's choice if higherIsBetter
            val = sign[ix] * value
            
            # Store user choice
            manual[ix] = val
            
            # Find the minimum weight on this dimension that achieves the chosen position
            # if user re-adjusts that means its important
            active = scores.values[:, ix] <= val  # larger = worse
            
            # Find the system that violates the other constraints the least
            # with no incentive to beat a constraint
                col = scores.values[:, p]
                limit = col[active].min()  # best feasible
                # whichever is worse of best feasible and requested
                thresh[p] = max(manual[p], limit)
                active &= (col <= thresh[p])  # filter

            for i in range(len(sliders)):
                if i != ix:
                    sliders[i].value = int(thresh[i]*sign[i])


            lock[0] = False

    for s in sliders:
        s.observe(limit, names='value')


    return widgets.VBox(sliders)

go()